In [1]:
from datasets import load_dataset

In [3]:
datasets = load_dataset('Yaxin/SemEval2016Task5Raw','restaurants_dutch')

Found cached dataset sem_eval2016_task5_raw (/home/eunjin/.cache/huggingface/datasets/Yaxin___sem_eval2016_task5_raw/restaurants_dutch/0.0.1/c28216cf58dd6bcb347f4819d7f8f89c8cf976381797c7100cd46a530c8c3784)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
datasets['validation']['text'][5]

'Het ligt wel iets buiten het centrum , maar de wandeling wordt beloond met een zeer goede maaltijd .'

In [6]:
datasets['validation']['opinions'][5]

[{'category': 'LOCATION#GENERAL',
  'from': '0',
  'polarity': 'positive',
  'target': 'null',
  'to': '0'},
 {'category': 'FOOD#QUALITY',
  'from': '90',
  'polarity': 'positive',
  'target': 'maaltijd',
  'to': '98'}]

In [9]:
import pandas as pd

In [10]:
def make_temp_review(sent, start, end):
    sent += ' '
    end_idx = 0
    while sent[end+end_idx] != ' ':
        end_idx += 1
    return sent[:start] +'[SEP]'+ sent[start:end+end_idx] + '[SEP]' + sent[end+end_idx:]


In [11]:
def make_bioes(sent, tags, pol):
    b=False
    for i , t in enumerate(sent.split()):
        if tags[i] != 'O':
            continue
        if t[:5]=='[SEP]' and t[-5:]=='[SEP]':
            tags[i]=f'S-{pol}'
        elif t[:5]=='[SEP]' and t[-5:]!='[SEP]':
            tags[i]=f'B-{pol}'
            b=True
        elif t[:5]!='[SEP]' and t[-5:]=='[SEP]':
            tags[i]=f'E-{pol}'
            b=False
        elif  t[:5]!='[SEP]' and t[-5:]!='[SEP]' and b:
            tags[i]=f'I-{pol}'
        else:
            tags[i]='O'
    return tags

In [12]:
def process_data(lang):
    dataset = load_dataset('Yaxin/SemEval2016Task5Raw',f'restaurants_{lang}')
    for mode in ['train','validation','test']:
        data = dataset[mode]
        new_df = pd.DataFrame(columns=['review','target','polarity', 'tags'])
        review= []
        targets = []
        pols = []
        tags = []
        for i, row in enumerate(data['opinions']):
            row = sorted(row, key=lambda x: x['from'])
            temp_from=[]
            temp_to=[]
            temp_target = []
            temp_pol = []
            for j in range(len(row)):
                if row[j]['target']!='NULL' and row[j]['target']!='null':
                    if int(row[j]['from']) in temp_from:
                        continue
                    temp_from.append(int(row[j]['from']))
                    temp_to.append(int(row[j]['to']))
                    temp_target.append(row[j]['target'])
                    temp_pol.append(row[j]['polarity'])
            
            if temp_target:
                temp_tags = ['O' for _ in range(len(data['text'][i].split()))]
                for k, p in enumerate(temp_pol):
                    temp_sent = make_temp_review( data['text'][i], temp_from[k],temp_to[k])
                    temp_tags = make_bioes(temp_sent.strip(), temp_tags, p)
                tags.append(' '.join(temp_tags))
                targets.append('<&>'.join(temp_target))
                pols.append('<&>'.join(temp_pol))
                review.append(data['text'][i])
        
        new_df['tags']=tags
        new_df['review']=review
        new_df['target']=targets
        new_df['polarity']=pols
        new_df.to_csv(f'datasets/res_{lang}_{mode}.tsv',sep='\t',index=False)
        print('DONE in {}'.format(mode))

In [13]:
process_data('dutch')

Found cached dataset sem_eval2016_task5_raw (/home/eunjin/.cache/huggingface/datasets/Yaxin___sem_eval2016_task5_raw/restaurants_dutch/0.0.1/c28216cf58dd6bcb347f4819d7f8f89c8cf976381797c7100cd46a530c8c3784)


  0%|          | 0/3 [00:00<?, ?it/s]

DONE in train
DONE in validation
DONE in test


In [14]:
for lang in ['english','french','spanish','russian']:
    process_data(lang)
    print('----{}----'.format(lang))

Found cached dataset sem_eval2016_task5_raw (/home/eunjin/.cache/huggingface/datasets/Yaxin___sem_eval2016_task5_raw/restaurants_english/0.0.1/c28216cf58dd6bcb347f4819d7f8f89c8cf976381797c7100cd46a530c8c3784)


  0%|          | 0/3 [00:00<?, ?it/s]

DONE in train
DONE in validation
DONE in test
----english----


Found cached dataset sem_eval2016_task5_raw (/home/eunjin/.cache/huggingface/datasets/Yaxin___sem_eval2016_task5_raw/restaurants_french/0.0.1/c28216cf58dd6bcb347f4819d7f8f89c8cf976381797c7100cd46a530c8c3784)


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

DONE in train
DONE in validation
DONE in test
----spanish----


Found cached dataset sem_eval2016_task5_raw (/home/eunjin/.cache/huggingface/datasets/Yaxin___sem_eval2016_task5_raw/restaurants_russian/0.0.1/c28216cf58dd6bcb347f4819d7f8f89c8cf976381797c7100cd46a530c8c3784)


  0%|          | 0/3 [00:00<?, ?it/s]

DONE in train
DONE in validation
DONE in test
----russian----


In [17]:
def new_tagging(lang):
    
    for mode in ['train','validation']:
        en_df = pd.read_csv(f'datasets/res_english_{mode}.tsv',sep='\t')
        target_df = pd.read_csv(f'datasets/res_english_{mode}_in_{lang}.tsv',sep='\t')
        target_df['tags'] = en_df['tags'].tolist()
        target_df.to_csv(f'datasets/res_english_{mode}_in_{lang}.tsv',sep='\t',index=False)
        print('DONE in {}'.format(mode))

In [18]:
for lang in ['french','spanish','russian','dutch']:
    new_tagging(lang)
    print('----{}----'.format(lang))

DONE in train
DONE in validation
----french----
DONE in train
DONE in validation
----spanish----
DONE in train
DONE in validation
----russian----
DONE in train
DONE in validation
----dutch----
